In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !pip install -r requirements.txt

In [6]:
import pandas as pd
import numpy as np
import ast
import utils
from functools import partial

In [7]:
from utils import get_distance_in_miles_to_point
CAMBRIDGE = (42.3736, -71.1097)
partial(get_distance_in_miles_to_point, center=CAMBRIDGE)

functools.partial(<function get_distance_in_miles_to_point at 0x11f958c10>, center=(42.3736, -71.1097))

In [55]:
# df_tip = pd.read_json('yelp_dataset/yelp_academic_dataset_tip.json', lines=True)
# df_business = pd.read_json('yelp_dataset/yelp_academic_dataset_business.json', lines=True)

In [35]:
%%time
CAMBRIDGE = (42.3736, -71.1097)
df_close = utils.load_close_businesses_and_process_and_save(CAMBRIDGE, 'cambridge', 10)

CPU times: user 1.3 s, sys: 14.5 ms, total: 1.31 s
Wall time: 1.31 s


In [36]:
df_close.categories

0        [Specialty Schools,  Massage Schools,  Middle ...
1              [Restaurants,  Shopping,  Shopping Centers]
2        [Sandwiches,  Food,  Restaurants,  Breakfast &...
3        [Health & Medical,  Beauty & Spas,  Massage Th...
4                              [Playgrounds,  Active Life]
                               ...                        
26148    [Auto Detailing,  Car Stereo Installation,  Au...
26149    [Community Service/Non-Profit,  Elementary Sch...
26150                [Japanese,  Sushi Bars,  Restaurants]
26151                      [Restaurants,  Pizza,  Italian]
26152                        [Automotive,  Auto Detailing]
Name: categories, Length: 26153, dtype: object

In [43]:
%%time
df_boba = utils.filter_df_with_categories(df_close, 'Bubble Tea')
df_pizza = utils.filter_df_with_categories(df_close, 'Pizza')

CPU times: user 20.3 ms, sys: 1.55 ms, total: 21.8 ms
Wall time: 20.5 ms


In [44]:
df_pizza[:5]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,distance_to_cambridge
23,L-Stk9lkVfPSFQlrFukCwA,Bill's Pizzeria,753 Beacon St,Newton,MA,02459,42.330774,-71.191639,3.5,205,1,"{'RestaurantsTakeOut': 'True', 'HasTV': 'True'...","[Pizza, Restaurants, Food Delivery Services,...","{'Monday': '9:0-22:0', 'Tuesday': '10:0-22:0',...",5.131761
83,4YpDKbN5fjeduPVxE1f2Kw,Brother's Pizza House,186 Cambridge Rd,Woburn,MA,01801,42.468564,-71.178455,2.5,37,1,"{'RestaurantsReservations': 'False', 'Business...","[Pizza, Restaurants]","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",7.438224
150,7TUhzfY4gMnfjG6qwQZ9ag,Moody Street Pizzeria & Grill,934 Moody St,Waltham,MA,02453,42.360263,-71.239288,2.5,11,0,"{'RestaurantsAttire': ''casual'', 'BusinessAcc...","[Pizza, Restaurants]",,6.696422
166,JpnTfmWmeo0YQEEFdam_MQ,Cook's Window Pizza,19 Washington St,Wellesley,MA,02481,42.325426,-71.260068,3.5,27,1,"{'BusinessAcceptsCreditCards': 'True', 'GoodFo...","[Pizza, Food, Salad, Restaurants, Caterers...","{'Monday': '0:0-0:0', 'Tuesday': '10:30-17:0',...",8.385989
268,dYezyZvEPYK0HRmgPxc5Ng,Croma,269 Newbury St,Boston,MA,02228,42.349557,-71.083595,3.5,99,0,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","[Pizza, Restaurants, Italian]",,2.130628


In [296]:
df_close.merge(df_tip, on = 'business_id').to_csv('data/tips_within_10_miles_of_cambridge.csv', index = False)

In [257]:
df_boba.sort_values('stars', ascending=False)
df_boba.sort_values('distance_to_cambridge')

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,distance_to_cambridge
6797,6nrH9eONhS8UzHIaLYPEuw,Limoo Tea Bar,1032 Commonwealth Ave,Boston,MA,02215,42.351558,-71.122221,4.0,150,0,"{'BusinessParking': '{'garage': False, 'street...","[Bubble Tea, Restaurants, Mediterranean, Co...","{'Monday': '15:0-23:0', 'Tuesday': '15:0-23:0'...",1.650880
19861,RzcDKKBkxy1JPCEFoGrV8A,Gong Cha,100 Cambridgeside Pl,Cambridge,MA,02141,42.368436,-71.076129,3.5,32,1,"{'BusinessAcceptsCreditCards': 'True', 'Busine...","[Bubble Tea, Restaurants, Food Court, Food,...","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'...",1.754776
14511,QbG-Ju21BhbPaNztMx8p1w,Dolphin Bay,72 Brighton Ave,Boston,MA,02134,42.352460,-71.128802,4.5,486,1,"{'GoodForMeal': '{'dessert': True, 'latenight'...","[Bubble Tea, Food, Taiwanese, Restaurants, ...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ...",1.756488
16975,-gbgBe9mxp7uFIUjrc1yxQ,Moon Flower House,110 Brighton Ave,Boston,MA,02134,42.352653,-71.130420,4.5,121,1,"{'Alcohol': 'u'none'', 'BusinessAcceptsCreditC...","[Bubble Tea, Desserts, Burgers, Food, Rest...","{'Monday': '12:0-22:30', 'Tuesday': '12:0-22:3...",1.793114
7693,rbVhmygaGIEKNODTIYp9AQ,One Zo,145 Brighton Ave,Allston,MA,02134,42.353140,-71.131659,4.0,14,1,{'RestaurantsTakeOut': 'None'},"[Bubble Tea, Food, Desserts]",,1.804884
24855,v972-kNc67HyGqdpqoCHSQ,Gong Cha,154 Harvard Ave,Allston,MA,02134,42.351703,-71.131533,3.5,105,1,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","[Bubble Tea, Food, Taiwanese, Restaurants]","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",1.879708
15223,go6pZHE2fCXFdO-ue1bH1g,Tealosophy,3 N Beacon St,Boston,MA,02134,42.353769,-71.137503,4.0,95,0,"{'BikeParking': 'True', 'RestaurantsTakeOut': ...","[Bubble Tea, Juice Bars & Smoothies, Shaved ...","{'Monday': '12:0-22:0', 'Tuesday': '12:0-22:0'...",1.974531
21688,bmXUFGarMFaZ-EWRlNg0Pg,Ten One Tea House,279 Newbury St,Boston,MA,02116,42.349496,-71.084086,4.0,74,1,"{'NoiseLevel': 'u'average'', 'BikeParking': 'T...","[Bubble Tea, Food]","{'Monday': '12:0-20:0', 'Tuesday': '12:0-20:0'...",2.118252
17237,JlEJ3-pZwNvblzpqqSIYdA,Cha Feo,248 Newbury St,Boston,MA,02116,42.349361,-71.082531,4.5,18,1,"{'RestaurantsTakeOut': 'True', 'OutdoorSeating...","[Bubble Tea, Cafes, Restaurants, Food, Cof...","{'Monday': '12:0-20:0', 'Tuesday': '12:0-20:0'...",2.175609
19900,FVS5EJ9-sqZzJ0enRAs2jw,POKE by Love Art,103 Beverly St,Boston,MA,02114,42.365524,-71.059475,4.0,25,1,"{'RestaurantsTableService': 'False', 'Wheelcha...","[Bubble Tea, Hawaiian, Sushi Bars, Restaura...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ...",2.630357


In [266]:
df_boba_tips = df_boba.merge(df_tip, on='business_id')

In [268]:
df_boba_tips.sort_values('compliment_count')

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,distance_to_cambridge,user_id,text,date,compliment_count
0,bwZlXQ3FPFWmewJupcOcAQ,Tsaocaa,10A Tyler St,Boston,MA,02111,42.350978,-71.060478,3.5,23,1,"{'BusinessAcceptsCreditCards': 'True', 'NoiseL...","[Bubble Tea, Juice Bars & Smoothies, Restaur...","{'Monday': '12:0-20:30', 'Tuesday': '12:0-20:3...",2.964133,SmY61ft-dwkgSuT-vtg7zg,They have a $10 credit card minimum!,2020-03-08 17:44:23,0
223,go6pZHE2fCXFdO-ue1bH1g,Tealosophy,3 N Beacon St,Boston,MA,02134,42.353769,-71.137503,4.0,95,0,"{'BikeParking': 'True', 'RestaurantsTakeOut': ...","[Bubble Tea, Juice Bars & Smoothies, Shaved ...","{'Monday': '12:0-22:0', 'Tuesday': '12:0-22:0'...",1.974531,_P_0piqDhfbKCtpnR3ozMw,Very nice shaved snow and best pearl milk tea ...,2016-09-28 20:12:18,0
222,go6pZHE2fCXFdO-ue1bH1g,Tealosophy,3 N Beacon St,Boston,MA,02134,42.353769,-71.137503,4.0,95,0,"{'BikeParking': 'True', 'RestaurantsTakeOut': ...","[Bubble Tea, Juice Bars & Smoothies, Shaved ...","{'Monday': '12:0-22:0', 'Tuesday': '12:0-22:0'...",1.974531,hKNkXnJJnRBqMJco5TuVQw,They have shaved ice!!!,2014-12-05 00:29:46,0
221,go6pZHE2fCXFdO-ue1bH1g,Tealosophy,3 N Beacon St,Boston,MA,02134,42.353769,-71.137503,4.0,95,0,"{'BikeParking': 'True', 'RestaurantsTakeOut': ...","[Bubble Tea, Juice Bars & Smoothies, Shaved ...","{'Monday': '12:0-22:0', 'Tuesday': '12:0-22:0'...",1.974531,9YvNBNbdv3rUajIQYWhS-A,A good place to hang out with friends. Tasty s...,2015-07-08 23:06:17,0
220,go6pZHE2fCXFdO-ue1bH1g,Tealosophy,3 N Beacon St,Boston,MA,02134,42.353769,-71.137503,4.0,95,0,"{'BikeParking': 'True', 'RestaurantsTakeOut': ...","[Bubble Tea, Juice Bars & Smoothies, Shaved ...","{'Monday': '12:0-22:0', 'Tuesday': '12:0-22:0'...",1.974531,sqTB8nQqe8dCs9DRseeA4A,customer wifi code,2016-03-10 23:23:27,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,7gymzvjJcwIFOUYF1UopaQ,The Juice Bar,"40-44 Harrison Ave, Unit 1A",Boston,MA,02111,42.351611,-71.061191,4.0,159,0,"{'WiFi': 'u'no'', 'BusinessAcceptsCreditCards'...","[Bubble Tea, Food, Juice Bars & Smoothies, ...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",2.910175,GNcNd4UiCQIe_IFmazNcIw,Watermelon juice is tasty. Make sure the girl ...,2013-05-01 13:33:53,0
105,7gymzvjJcwIFOUYF1UopaQ,The Juice Bar,"40-44 Harrison Ave, Unit 1A",Boston,MA,02111,42.351611,-71.061191,4.0,159,0,"{'WiFi': 'u'no'', 'BusinessAcceptsCreditCards'...","[Bubble Tea, Food, Juice Bars & Smoothies, ...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",2.910175,KFaUuuvfw-6rAbTzuYwU0g,3.75 for quickly quality milk tea. Cash only. ...,2015-01-28 18:55:32,0
104,7gymzvjJcwIFOUYF1UopaQ,The Juice Bar,"40-44 Harrison Ave, Unit 1A",Boston,MA,02111,42.351611,-71.061191,4.0,159,0,"{'WiFi': 'u'no'', 'BusinessAcceptsCreditCards'...","[Bubble Tea, Food, Juice Bars & Smoothies, ...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",2.910175,i6kNeNRrER8ocnpkBJ3wBA,"Had a really great warm taro tea, one of the b...",2011-06-13 00:14:58,0
103,7gymzvjJcwIFOUYF1UopaQ,The Juice Bar,"40-44 Harrison Ave, Unit 1A",Boston,MA,02111,42.351611,-71.061191,4.0,159,0,"{'WiFi': 'u'no'', 'BusinessAcceptsCreditCards'...","[Bubble Tea, Food, Juice Bars & Smoothies, ...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",2.910175,zgP0lQWF5b_76WhFHjNidg,$3.75 for bubble tea. Open later than most,2012-07-06 01:33:09,0


In [93]:
df_close.sort_values('distance_to_cambridge')

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,distance_to_cambridge
4347,tBSH5aDPtc2FOQQ53hThbw,Desinvolte,"30 JFK St, Lower Level",Cambridge,MA,02138,42.373616,-71.109734,4.5,5,0,"{'RestaurantsPriceRange2': '3', 'BusinessAccep...","[Women's Clothing, Shopping, Fashion]",,0.002032
15419,sbnKzkgWcmlRRKG7-2BaUw,Ritz Camera Centers,100 Cambridgeside Pl Ofc,Cambridge,MA,02141,42.373616,-71.109734,4.0,7,0,"{'BusinessParking': '{'garage': False, 'street...","[Photography Stores & Services, Shopping]",,0.002032
4291,M8I5OL8mMHaYkmNP1pUHQg,MIT Game Room,"84 Massachusetts Ave, First Fl",Cambridge,MA,02238,42.373616,-71.109734,3.5,6,0,"{'Alcohol': ''none'', 'BusinessAcceptsCreditCa...","[Pool Halls, Nightlife, Local Flavor]",,0.002032
13806,14bHRyUmxvPzmUQD3BGrOA,New York & Company,100 Cambridgeside Pl Ofc,Cambridge,MA,02141,42.373616,-71.109734,4.5,8,0,"{'BusinessAcceptsCreditCards': 'True', 'Busine...","[Shopping, Fashion]",,0.002032
13949,aoDGdIuFVU3JoaaACAKtyg,Seamless Transition,1 Broadway,Cambridge,MA,02139,42.373711,-71.109776,1.5,8,1,{'BusinessAcceptsCreditCards': 'True'},"[Apartments, Real Estate, Home Services]",,0.008597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14742,fvk0Khny0_-VsG8vP3eA2A,Tile International,"319 Waverley Oaks Rd, Ste 1",Waltham,MA,02452,42.385789,-71.205987,3.5,10,1,"{'BusinessAcceptsCreditCards': 'True', 'Busine...","[Building Supplies, Home Services, Flooring,...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...",4.998697
5921,DUHuijtSH7IpcQD3_oD2YA,The Ultimate Self Defense & Performance Center,1663 Columbia Rd,South Boston,MA,02127,42.331111,-71.030628,5.0,5,1,{'GoodForKids': 'True'},"[Trainers, Martial Arts, Self-defense Classe...","{'Monday': '8:30-21:30', 'Tuesday': '6:30-21:3...",4.998844
6670,FtRAFhow4YLjRjkY2WuTDQ,Shaker Workshops,18 Mill Ln,Arlington,MA,02476,42.425370,-71.178038,1.5,8,0,"{'BusinessAcceptsCreditCards': 'True', 'Busine...","[Shopping, Home Decor, Furniture Stores, Ho...","{'Monday': '10:0-17:0', 'Tuesday': '10:0-17:0'...",4.999046
3447,JgDsrE1e92nYm7zVWbPF_w,I Love Kickboxing - Waltham,87 Beaver St,Waltham,MA,02453,42.382520,-71.206650,4.5,13,0,"{'ByAppointmentOnly': 'False', 'BusinessParkin...","[Active Life, Gyms, Martial Arts, Weight Lo...","{'Monday': '17:0-20:30', 'Tuesday': '17:0-20:3...",4.999493


In [288]:
df_reviews = pd.read_json('yelp_dataset/yelp_academic_dataset_review.json', lines=True)

In [290]:
df_reviews_close = df_close.merge(df_reviews, on = 'business_id')

In [294]:
df_reviews.shape

(8635403, 9)

In [297]:
df_reviews_close.to_csv('data/reviews_within_10_miles_of_cambridge.csv', index=False)

In [340]:
df_reviews_boba = df_reviews_close.iloc[[i for i, x in enumerate(df_reviews_close.business_id) if x in list(df_boba.business_id)]]

In [348]:
df_reviews.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')

In [354]:
df_reviews.iloc[:,2:].sample(10)

,business_id,stars,useful,funny,cool,text,date
5312613,g1Qok-8QvCHuZV0HL0DX5w,3,1,0,0,Bloody Mary was just okay. Not the best one I'...,2015-02-18 17:24:21
7740857,mLOGZ8ayN2imj8kO35Jz4w,1,0,0,0,"Service, quality, everything has went downhill...",2020-11-24 01:57:24
1926330,EkYQgWOYrM52rMyfrEMczQ,4,1,0,0,This place was nice but wouldn't consider it t...,2010-07-23 13:21:21
1794618,CF8EjfGgyTMIR7klsl3flQ,1,6,0,0,The most incompetent people in the world work ...,2008-12-26 20:43:54
8333866,J2oWzpLVYqj0XOPVD1L2dA,5,0,0,0,"Taste so good, I really like their bbq, it's o...",2016-03-27 17:46:58
119410,mvB_Z5NbYPLRvUNPt3v06w,3,1,0,0,Very good sandwich or wrap. Pizza is good. Mor...,2017-08-20 01:54:04
7793541,_Dl1YmHct9J0Nn7KsbTW_w,5,0,0,0,"Friendly staff, fast service as always. I alwa...",2018-04-15 14:57:14
2504611,tqUVKxmb3R4BM1hf_Hy9nQ,1,0,0,0,Ordered Chow Mein and they gave me maybe 95% g...,2019-01-31 23:40:21
440338,k7-oClu97qColM6BcCaQMw,4,1,0,0,This place has a great atmosphere. The servers...,2010-10-01 19:56:37
926737,b7kKPiS9fJLpnScMs4qAnQ,5,0,0,0,Great Staff Great Food and a Fun atmosphere a...,2013-01-06 04:42:10


In [369]:
_to_exclude = ['review_id', 'postal_code', 'date', 
               'attributes', 'address', 'business_id', 
               'categories', 'user_id', 'stars_x', 'stars_y',
              'latitude', 'longitude', 'is_open'
              'cool', 'funny']
cols = list(set(df_reviews_boba.columns) - set(_to_exclude))
df_reviews_boba[cols].sample(10)

,name,city,is_open,distance_to_cambridge,state,review_count,hours,useful,text,cool
1635935,Gong Cha,Allston,1,1.879708,MA,105,"{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",0,"I ordered a Strawberry Matcha Latte with boba,...",0
515715,Chatime,Boston,0,2.899948,MA,236,"{'Tuesday': '12:0-18:0', 'Wednesday': '12:0-18...",2,Bought a drink here and after taking one sip i...,0
1593439,Monkey King Tea,Malden,1,3.670842,MA,244,"{'Wednesday': '11:30-20:0', 'Thursday': '11:30...",17,This place is too bomb! I fell in love right w...,10
515583,Chatime,Boston,0,2.899948,MA,236,"{'Tuesday': '12:0-18:0', 'Wednesday': '12:0-18...",0,ChaTime is my staple boba tea place to go when...,0
948907,Dolphin Bay,Boston,1,1.756488,MA,486,"{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ...",0,This is probably my favorite Taiwanese restaur...,0
994068,Tealosophy,Boston,0,1.974531,MA,95,"{'Monday': '12:0-22:0', 'Tuesday': '12:0-22:0'...",0,Good place to have snow shaved ice! They are r...,0
788366,Reign Drink Lab,Dorchester,1,5.456567,MA,104,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-20:0', '...",1,We LOVE Reign. Everything is consistently abso...,1
515647,Chatime,Boston,0,2.899948,MA,236,"{'Tuesday': '12:0-18:0', 'Wednesday': '12:0-18...",5,"Legit thick, creamy real taro milk tea! None o...",1
980955,Happy Lemon Boston,Boston,1,2.800257,MA,157,"{'Monday': '11:0-19:0', 'Tuesday': '11:0-19:0'...",0,"So nice taste, and it's my favorite milk tea i...",0
762928,Gong Cha,Malden,1,4.090146,MA,27,"{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",1,"Saw this place was new and open as I drove by,...",0


In [372]:
df_boba_tips.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours', 'distance_to_cambridge', 'user_id',
       'text', 'date', 'compliment_count'],
      dtype='object')

In [376]:
_to_exclude = ['postal_code', 'date', 'hours',
               'attributes', 'address', 'business_id', 
               'categories', 'user_id', 'stars_x', 'stars_y',
              'latitude', 'longitude', 'is_open'
               'funny']
cols = list(set(df_boba_tips.columns) - set(_to_exclude))
df_boba_tips[cols].sample(10)

,name,stars,city,is_open,distance_to_cambridge,state,review_count,compliment_count,text
153,Dolphin Bay,4.5,Boston,1,1.756488,MA,486,0,Yummy food for reasonable price!!!
244,Moon Flower House,4.5,Boston,1,1.793114,MA,121,0,很好吃！环境也很干净舒服，终于找到奥尔良烤翅太开心了吧。还是那个熟悉的配方，照烧鸡腿堡，新奥...
205,Happy Lemon Boston,4.0,Boston,1,2.800257,MA,157,0,"The check in is 10% off, not $2 off"
28,Chatime,3.0,Boston,0,2.899948,MA,236,0,It is officially opened! There is a buy one ge...
96,The Juice Bar,4.0,Boston,0,2.910175,MA,159,0,Love the jasmine tea!!
50,Chatime,3.0,Boston,0,2.899948,MA,236,0,Accepts credit card--no minimum.
172,Dolphin Bay,4.5,Boston,1,1.756488,MA,486,0,Authentic Taiwanese food! Reasonable price and...
146,Dolphin Bay,4.5,Boston,1,1.756488,MA,486,0,Kicking ass MTF awesome Taiwanese food with th...
6,Pholicious,3.5,Saugus,0,7.481981,MA,21,0,"If you don't want bubbles in your bubble tea, ..."
154,Dolphin Bay,4.5,Boston,1,1.756488,MA,486,0,Rice dishes are mediocre and over-priced given...
